In [2]:
# 1. 기본 모듈 임포트 및 .env 경로 지정
import pandas as pd
import os
from dotenv import load_dotenv

# .env 경로 지정 (노트북이 mlops_team 폴더 안에 있을 경우)
dotenv_path = '../../.env' 
load_dotenv(dotenv_path=dotenv_path)

True

In [5]:
# 2. S3에서 읽어올 파일 경로 구성
s3_bucket_name = "mlops-prj"
s3_file_key = "data/weather/realtime_hourly_raw/year=2025/month=06/day=02/hour=17/data.csv"

s3_full_path = f"s3://{s3_bucket_name}/{s3_file_key}"

print(f"S3에서 다음 파일을 읽어옵니다: {s3_full_path}")

S3에서 다음 파일을 읽어옵니다: s3://mlops-prj/data/weather/realtime_hourly_raw/year=2025/month=06/day=02/hour=17/data.csv


In [10]:

!pip install "botocore>=1.38.27,<1.39.0"
!pip install --upgrade boto3 s3transfer

  Using cached botocore-1.38.27-py3-none-any.whl.metadata (5.7 kB)
Using cached botocore-1.38.27-py3-none-any.whl (13.6 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.37.3
    Uninstalling botocore-1.37.3:
      Successfully uninstalled botocore-1.37.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.22.0 requires botocore<1.37.4,>=1.37.2, but you have botocore 1.38.27 which is incompatible.


In [11]:
# 3. CSV 파일 읽기 시도
try:
    df_from_s3 = pd.read_csv(s3_full_path, encoding='utf-8-sig')  # storage_options 생략 가능
    print("[성공] S3에서 CSV 파일을 성공적으로 읽었습니다!")
except FileNotFoundError:
    print(f"[오류] 파일을 찾을 수 없습니다: {s3_full_path}")
except pd.errors.EmptyDataError:
    print(f"[오류] 파일이 비어있습니다: {s3_full_path}")
except Exception as e:
    print(f"[오류] S3에서 파일을 읽는 중 문제가 발생했습니다: {e}")

[성공] S3에서 CSV 파일을 성공적으로 읽었습니다!


In [ ]:
# 4. DataFrame 정보 요약
print("\n--- df.info() ---")
df_from_s3.info()

print("\n--- df.describe(include='all') ---")
print(df_from_s3.describe(include='all'))


print(f"\n--- df.shape ---")
print(f"행: {df_from_s3.shape[0]}, 열: {df_from_s3.shape[1]}")


--- df.info() ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 46 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ObservationTime      1 non-null      object 
 1   StationID            1 non-null      int64  
 2   WindDirection        1 non-null      int64  
 3   WindSpeed            1 non-null      float64
 4   GustDirection        0 non-null      float64
 5   GustSpeed            0 non-null      float64
 6   GustTime             0 non-null      float64
 7   LocalPressure        1 non-null      float64
 8   SeaLevelPressure     1 non-null      float64
 9   PressureTrend        0 non-null      float64
 10  PressureChange       0 non-null      float64
 11  Temperature          1 non-null      float64
 12  DewPointTemperature  1 non-null      float64
 13  RelativeHumidity     1 non-null      float64
 14  VaporPressure        1 non-null      float64
 15  HourlyRainfall       1 no

In [14]:
df_from_s3.columns

Index(['ObservationTime', 'StationID', 'WindDirection', 'WindSpeed',
       'GustDirection', 'GustSpeed', 'GustTime', 'LocalPressure',
       'SeaLevelPressure', 'PressureTrend', 'PressureChange', 'Temperature',
       'DewPointTemperature', 'RelativeHumidity', 'VaporPressure',
       'HourlyRainfall', 'DailyRainfall', 'CumulativeRainfall',
       'RainfallIntensity', 'SnowDepth3Hr', 'DailySnowDepth', 'TotalSnowDepth',
       'CurrentWeatherCode', 'PastWeatherCode', 'WeatherCode',
       'TotalCloudCover', 'MidLowCloudCover', 'LowestCloudHeight', 'CloudType',
       'UpperCloudType', 'MidCloudType', 'LowCloudType', 'Visibility',
       'SunshineDuration', 'SolarRadiation', 'GroundCondition',
       'GroundTemperature', 'SoilTemperature5cm', 'SoilTemperature10cm',
       'SoilTemperature20cm', 'SoilTemperature30cm', 'SeaCondition',
       'WaveHeight', 'MaxWindForce', 'PrecipitationData', 'ObservationType'],
      dtype='object')

In [15]:

print("\n--- df.head() ---")
print(df_from_s3.head())


--- df.head() ---
       ObservationTime  StationID  WindDirection  WindSpeed  GustDirection  \
0  2025-06-02 17:00:00        159             20        3.8            NaN   

   GustSpeed  GustTime  LocalPressure  SeaLevelPressure  PressureTrend  ...  \
0        NaN       NaN          995.2            1003.2            NaN  ...   

   GroundTemperature  SoilTemperature5cm  SoilTemperature10cm  \
0               21.5                22.2                 21.1   

   SoilTemperature20cm  SoilTemperature30cm  SeaCondition  WaveHeight  \
0                 19.7                 19.5           NaN         0.5   

   MaxWindForce  PrecipitationData  ObservationType  
0           NaN                  4                1  

[1 rows x 46 columns]


# 16시 데이터 받아서 17시랑 비교해보자비교해보자

In [ ]:
# 16시 데이터
s3_bucket_name = "mlops-prj"
s3_file_key = "data/weather/realtime_hourly_raw/year=2025/month=06/day=02/hour=16/data.csv"

s3_full_path = f"s3://{s3_bucket_name}/{s3_file_key}"

print(f"S3에서 다음 파일을 읽어옵니다: {s3_full_path}")

S3에서 다음 파일을 읽어옵니다: s3://mlops-prj/data/weather/realtime_hourly_raw/year=2025/month=06/day=02/hour=16/data.csv


In [17]:
# 3. CSV 파일 읽기 시도
try:
    df_from_s3 = pd.read_csv(s3_full_path, encoding='utf-8-sig')  # storage_options 생략 가능
    print("[성공] S3에서 CSV 파일을 성공적으로 읽었습니다!")
except FileNotFoundError:
    print(f"[오류] 파일을 찾을 수 없습니다: {s3_full_path}")
except pd.errors.EmptyDataError:
    print(f"[오류] 파일이 비어있습니다: {s3_full_path}")
except Exception as e:
    print(f"[오류] S3에서 파일을 읽는 중 문제가 발생했습니다: {e}")

[성공] S3에서 CSV 파일을 성공적으로 읽었습니다!


In [18]:
print("\n--- df.head() ---")
print(df_from_s3.head())


--- df.head() ---
       ObservationTime  StationID  WindDirection  WindSpeed  GustDirection  \
0  2025-06-02 16:00:00        159             23        2.8            NaN   

   GustSpeed  GustTime  LocalPressure  SeaLevelPressure  PressureTrend  ...  \
0        NaN       NaN            NaN               NaN            NaN  ...   

   GroundTemperature  SoilTemperature5cm  SoilTemperature10cm  \
0                NaN                 NaN                  NaN   

   SoilTemperature20cm  SoilTemperature30cm  SeaCondition  WaveHeight  \
0                  NaN                  NaN           NaN         NaN   

   MaxWindForce  PrecipitationData  ObservationType  
0           NaN                NaN              NaN  

[1 rows x 46 columns]
